# Reverting the BIOS to default values

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>reset your bios to default settings!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

<div class="alert alert-block alert-success">
<b> Revert BIOS to default: <br> </b> 
    <ul>
    <li> first we need to login to iLO servers so we need <i>system URL, Username,Password</i> with all these obtained we log into iLO servers.
    <li> And then we <i>GET</i> the place (directory) that we'll be working on. get_resource_directory module takes care of that and if it doesn't work we manually dive and get the uri.
        <li> Once we obtain the directory. We navigate to BIOS property and perform our operation there.
        <li> And then we set the <b>Action</b>: to <i>Bios.ResetBios</i>, and  <b>ResetType</b>: to <i>default</i>
        <li>And the <i>POST</i> and post the update to the server</li>
        <li> Once it is set to default settings we get the response status as 200. If any problem with server we get 400.</li>
    </ul>
</div>

In [2]:

def bios_revert_default(_redfishobj):

    bios_reset_action_uri = None
    resource_instances = get_resource_directory(_redfishobj)

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the relevant URI
        
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
        bios_response = _redfishobj.get(bios_uri)
        bios_reset_action_uri = bios_response.obj['Actions']['#Bios.ResetBios']['target']
    else:
        #Use Resource directory to find the relevant URI
        
        for instance in resource_instances:
            if '#Bios.' in instance['@odata.type']:
                bios_uri = instance['@odata.id']
                bios_data = _redfishobj.get(bios_uri)
                bios_reset_action_uri = bios_data.obj['Actions']['#Bios.ResetBios']['target']
                break

    body = {'Action': 'Bios.ResetBios', 'ResetType':'default'}
    resp = _redfishobj.post(bios_reset_action_uri, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))


<div class="alert alert-block alert-success">
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE.<br>
    <ul>
        <li>Once you enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will  be logged in to the server</li>
        <li>If server is down or Unreachable we get "server not reachable" message!</li>
        <li>and Once everything is done you are automatically logged out with <b>.logout()<b> 😁. </li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = input("Enter password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
                                                                            
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    bios_revert_default(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

_HAVING PROBELM???_
Please check [here](https://hewlettpackard.github.io/python-ilorest-library/Examples.html#change-bios-setting)